In [1]:
import requests
from Levenshtein import ratio
import pandas as pd
from datetime import datetime, timedelta
import re
import time
import sys
import os
from joblib import dump, load
from tqdm import tqdm
from pathlib import Path


# Add parent directory to sys.path for config imports
parent_dir = str(Path().resolve().parent)
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from config.settings import API_KEYS, API_ENDPOINTS

print("📊 Solana DeFi Tracker - Data Collection")
print(f"Cache directory: {os.path.normpath('../data')}")
print(f"Collection timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

📊 Solana DeFi Tracker - Data Collection
Cache directory: ..\data
Collection timestamp: 2025-09-05 14:00:51


#### Verify Cache Directories

In [2]:
# Verify cache directories exist, create if missing
cache_dirs = ['../data/api_responses', '../data/processed', '../data/temp']
for cache_dir in cache_dirs:
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir, exist_ok=True)
        print(f"✅ Created cache directory: {cache_dir}")
    else:
        print(f"✅ Cache directory exists: {cache_dir}")

✅ Cache directory exists: ../data/api_responses
✅ Cache directory exists: ../data/processed
✅ Cache directory exists: ../data/temp


#### Helper Functions

In [3]:
def make_request(url, headers=None, params=None, max_retries=3, is_post=False):
    """Make API request with retry logic"""
    for attempt in range(max_retries):
        try:
            if is_post:
                response = requests.post(url, headers=headers, json=params, timeout=30)
            else:
                response = requests.get(url, headers=headers, params=params, timeout=30)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"❌ Attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)  # Exponential backoff
            else:
                return None

def save_raw_data(data, filename, description=""):
    """Save raw API response to api_responses directory"""
    filepath = os.path.normpath(f"../data/api_responses/{filename}")
    dump(data, filepath)
    print(f"💾 Saved raw: {description} → {filepath}")
    return filepath

def save_processed_data(data, filename, description=""):
    """Save processed data to processed directory"""
    filepath = os.path.normpath(f"../data/processed/{filename}")
    dump(data, filepath)
    print(f"💾 Saved processed: {description} → {filepath}")
    return filepath

def save_cache(data, filename, description=""):
    """Save cache data to temp directory"""
    filepath = os.path.normpath(f"../data/temp/{filename}")
    dump(data, filepath)
    print(f"💾 Saved cache: {description} → {filepath}")
    return filepath

def load_cache(filename):
    """Load cache data from temp directory"""
    filepath = os.path.normpath(f"../data/temp/{filename}")
    if os.path.exists(filepath):
        try:
            return load(filepath)
        except Exception as e:
            print(f"⚠️ Failed to load cache {filename}: {e}")
            return None
    return None

def format_currency(amount):
    """Format currency amount with appropriate units (K, M, B)"""
    if amount is None or amount == 0:
        return "$0"
    
    if amount >= 1_000_000_000:
        return f"${amount/1_000_000_000:.2f}B"
    elif amount >= 1_000_000:
        return f"${amount/1_000_000:.2f}M"
    elif amount >= 1_000:
        return f"${amount/1_000:.2f}K"
    else:
        return f"${amount:.2f}"

#### Step 1: Collect DefiLlama Protocol Data (TVL)

In [4]:
print("\n🔍 Collecting Solana Protocol TVL Data...")

def get_all_solana_tvl_data():
    """
    Collect TVL data for Solana DeFi protocols from DefiLlama, excluding CEX and CEX-related protocols.
    
    """
 
    # DefiLlama API endpoint for all protocols
    base_url = API_ENDPOINTS['defillama']['base_url']
    protocols_url = f"{base_url}/protocols"
    
    # Make the API request
    all_protocols = make_request(protocols_url)
    
    if not all_protocols:
        print("❌ DefiLlama API failed")
        return None
    
    print(f"✅ DefiLlama API working! Found {len(all_protocols)} total protocols")
    
    # Comprehensive list of CEX names to exclude
    cex_list = [
        'binance', 'bybit', 'coinbase', 'kraken', 'kucoin', 'okx',
        'crypto.com', 'crypto', 'bitfinex', 'huobi', 'htx', 'gate', 'gate.io',
        'mexc', 'bitget', 'gemini', 'bitstamp', 'bithumb', 'bitpanda',
        'bitmex', 'coinex', 'upbit', 'revolut', 'coindcx', 'bitflyer',
        'coincheck', 'bitbank', 'swissborg', 'deribit'
    ]
    
    # Filter for Solana DeFi protocols (excluding CEX and CEX-related)
    solana_protocols = []
    excluded_protocols = []
    
    for protocol in all_protocols:
        chains = protocol.get('chains', [])
        category = protocol.get('category', '').lower()
        name = protocol.get('name', '').lower()
        
        is_solana = (
            'Solana' in chains or 
            'solana' in chains or
            any('solana' in str(chain).lower() for chain in chains) or
            protocol.get('chain') == 'Solana'
        )
        
        # Exclude CEX and CEX-related protocols
        is_cex_related = (
            category == 'cex' or
            any(cex in name for cex in cex_list)
        )
        
        if is_solana and not is_cex_related:
            tvl_value = protocol.get('tvl') or 0
            
            solana_protocols.append({
                'name': protocol.get('name', 'Unknown'),
                'slug': protocol.get('slug', ''),
                'tvl': tvl_value,
                'chains': chains,
                'category': protocol.get('category', 'Unknown'),
                'change_1h': protocol.get('change_1h'),
                'change_1d': protocol.get('change_1d'),
                'change_7d': protocol.get('change_7d'),
                'mcap': protocol.get('mcap'),
                'symbol': protocol.get('symbol', ''),
                'url': protocol.get('url', ''),
                'description': protocol.get('description', ''),
                'gecko_id': protocol.get("coingeckoId"),
                'timestamp': datetime.now()
            })
        elif is_solana and is_cex_related:
            excluded_protocols.append((protocol.get('name'), category))
    
    if excluded_protocols:
        print(f"⚠️ Excluded {len(excluded_protocols)} CEX-related protocols: {', '.join([name for name, _ in excluded_protocols[:5]])}")
        # Save excluded protocols log to cache directory
        save_cache(excluded_protocols, f'excluded_protocols_{datetime.now().strftime("%Y%m%d_%H%M%S")}.joblib', 
                    "Excluded CEX protocols log")  
    
    if not solana_protocols:
        print("❌ No Solana DeFi protocols found")
        return None
    
    # Convert to DataFrame
    df = pd.DataFrame(solana_protocols)
    
    # Sort by TVL descending
    df = df.sort_values(by="tvl", ascending=False).reset_index(drop=True)
    
    print(f"🌟 Found {len(df)} Solana DeFi protocols:")
    
    # Calculate statistics
    total_tvl = df['tvl'].sum()
    active_protocols = (df['tvl'] > 0).sum()
    
    print(f"📊 Total Solana DeFi TVL: ${total_tvl:,.0f}")
    print(f"📈 Active protocols (TVL > 0): {active_protocols}/{len(df)}")
    
    # Display top protocols
    print(f"\n{'Rank':<5} {'Protocol':<25} {'TVL':<15} {'Category':<20} {'1d Change':<10}")
    print("=" * 85)
    
    for i, row in df.head(20).iterrows():
        tvl_formatted = format_currency(row['tvl'])
        change_1d = row['change_1d']
        change_str = f"{change_1d:+.1f}%" if change_1d is not None else "N/A"
        
        print(f"{i+1:<5} {row['name'][:24]:<25} {tvl_formatted:<15} "
              f"{row['category'][:19]:<20} {change_str:<10}")
    
    # Show category breakdown
    category_breakdown = (
        df.groupby("category")['tvl']
        .agg(['count', 'sum'])
        .rename(columns={'count': 'protocols', 'sum': 'total_tvl'})
        .sort_values(by="total_tvl", ascending=False)
    )
    
    print(f"\n📋 Category Breakdown:")
    print(f"{'Category':<25} {'Count':<8} {'Total TVL':<15}")
    print("-" * 50)
    
    for category, row in category_breakdown.head(10).iterrows():
        print(f"{category[:24]:<25} {row['protocols']:<8} {format_currency(row['total_tvl']):<15}")
    
    # Save DataFrame to joblib
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'solana_defi_tvl_{timestamp}.joblib'
    save_raw_data(df, filename, 'Solana DeFi protocols TVL data')
    print(f"\n💾 DataFrame saved to {filename}")
    
    return df

# Execute the collection
tvl_df = get_all_solana_tvl_data()

if tvl_df is not None:
    print(f"\n✅ Successfully collected TVL data for {len(tvl_df)} Solana DeFi protocols")
    print("📁 DataFrame saved to joblib file for further analysis")
    print(f"📊 Dataset includes {len(tvl_df)} protocols worth ${tvl_df['tvl'].sum():,.0f} in total TVL")
else:
    print("\n❌ Failed to collect TVL data")

print("\n" + "=" * 50)


🔍 Collecting Solana Protocol TVL Data...
✅ DefiLlama API working! Found 6366 total protocols
⚠️ Excluded 42 CEX-related protocols: Binance CEX, OKX, Bitfinex, Bybit, Gate
💾 Saved cache: Excluded CEX protocols log → ..\data\temp\excluded_protocols_20250905_140054.joblib
🌟 Found 252 Solana DeFi protocols:
📊 Total Solana DeFi TVL: $77,749,203,760
📈 Active protocols (TVL > 0): 220/252

Rank  Protocol                  TVL             Category             1d Change 
1     Lido                      $38.36B         Liquid Staking       -0.2%     
2     Jito Liquid Staking       $3.05B          Liquid Staking       -1.5%     
3     Portal                    $2.77B          Bridge               +1.4%     
4     Kamino Lend               $2.61B          Lending              +0.3%     
5     Sanctum Validator LSTs    $2.43B          Liquid Staking       +1.1%     
6     Jupiter Perpetual Exchan  $2.41B          Derivatives          +0.7%     
7     Raydium AMM               $2.32B          Dexs  

#### Step 2: Collect DefiLlama Revenue Data

In [5]:
print("\n🔍 Collecting Solana Protocol Revenue Data...")

def get_solana_revenue_data():
    """Collect REVENUE data from DefiLlama"""
    base_url = API_ENDPOINTS['defillama']['base_url']
    revenue_url = f"{base_url}/overview/fees/solana"
    
    #
    params = {
        'dataType': 'dailyRevenue', 
        'excludeTotalDataChart': 'true',
        'excludeTotalDataChartBreakdown': 'true'
    }
    
    data = make_request(revenue_url, params=params)
    
    if not data:
        print("❌ Solana Revenue API failed")
        return None
    
    print(f"✅ Collected Solana revenue data")
    print(f"Total protocols found: {len(data.get('protocols', []))}")
    
    protocols = data.get('protocols', [])
    if not protocols:
        print("No protocol data found.")
        return None
    
    # Sort protocols by total24h in descending order
    sorted_protocols = sorted(protocols, 
                            key=lambda x: x.get('total24h', 0) or 0, 
                            reverse=True)
    
    # Build DataFrame
    revenue_list = []
    for protocol in sorted_protocols:
        revenue_list.append({
            'protocol': protocol.get('name', 'Unknown'),
            'revenue_24h': protocol.get('total24h', 0),  
            'revenue_7d': protocol.get('total7d', 0),    
            'revenue_30d': protocol.get('total30d', 0),  
            'revenue_all_time': protocol.get('totalAllTime', 0),
            'data_type': 'revenue',  
            'chain': 'solana',
            'timestamp': datetime.now()
        })
    
    df = pd.DataFrame(revenue_list)
    
    # Display summary
    protocols_with_data = (df['revenue_24h'] > 0).sum()
    print(f"💵 Found revenue data for {protocols_with_data} active protocols:")
    
    # Show top 10
    top_protocols = df.sort_values(by="revenue_24h", ascending=False).head(10)
    print(f"\n{'Protocol':<25} {'24h Revenue':<15} {'7d Revenue':<15} {'30d Revenue':<15}")
    print("=" * 75)
    
    for _, row in top_protocols.iterrows():
        if row['revenue_24h'] > 0:
            print(f"{row['protocol'][:24]:<25} {format_currency(row['revenue_24h']):<15} "
                  f"{format_currency(row['revenue_7d']):<15} {format_currency(row['revenue_30d']):<15}")
    
    # Save DataFrame to joblib
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'solana_revenue_{timestamp}.joblib'
    save_raw_data(df, filename, 'Solana revenue data')
    print(f"\n💾 DataFrame saved to {filename}")
    
    return df


print("📊 Solana Protocol Revenue Tracker")
print("=" * 50)

revenue_df = get_solana_revenue_data()

if revenue_df is not None:
    print(f"\n✅ Successfully collected revenue data for {len(revenue_df)} protocols")
    print("📁 DataFrame saved to joblib file for further analysis")
else:
    print("\n❌ Failed to collect revenue data")

print("\n" + "=" * 50)


🔍 Collecting Solana Protocol Revenue Data...
📊 Solana Protocol Revenue Tracker
✅ Collected Solana revenue data
Total protocols found: 104
💵 Found revenue data for 91 active protocols:

Protocol                  24h Revenue     7d Revenue      30d Revenue    
pump.fun                  $1.62M          $10.26M         $45.90M        
Axiom                     $1.56M          $10.65M         $53.68M        
Jupiter Perpetual Exchan  $785.31K        $5.74M          $23.71M        
PumpSwap                  $541.92K        $2.16M          $6.39M         
Phantom Wallet            $500.40K        $3.48M          $16.79M        
Photon                    $159.93K        $1.05M          $5.94M         
Solana                    $146.25K        $1.06M          $4.56M         
Meteora DAMM V2           $124.96K        $895.45K        $6.17M         
Binance Staked SOL        $124.92K        $572.36K        $1.61M         
Raydium AMM               $96.78K         $803.08K        $3.58M         


#### Step 3: Collect Solana Fees Data

In [6]:
print("\n🔍 Collecting Solana Protocol Fees Data...")

def get_solana_fees_data():
    
    base_url = API_ENDPOINTS['defillama']['base_url']
    fees_url = f"{base_url}/overview/fees/solana"
    
  
    params = {
        'dataType': 'dailyFees',  
        'excludeTotalDataChart': 'true',
        'excludeTotalDataChartBreakdown': 'true'
    }
    
    data = make_request(fees_url, params=params)
    
    if not data:
        print("❌ Solana Fees API failed")
        return None
    
    print(f"✅ Collected Solana fees data")
    print(f"Total protocols found: {len(data.get('protocols', []))}")
    
    protocols = data.get('protocols', [])
    
    if not protocols:
        print("No protocol data found.")
        return None
    
    # Sort protocols by total24h in descending order
    sorted_protocols = sorted(protocols, 
                            key=lambda x: x.get('total24h', 0) or 0, 
                            reverse=True)
    
    # Build list for DataFrame
    fees_list = []
    for protocol in sorted_protocols:
        fees_list.append({
            'protocol': protocol.get('name', 'Unknown'),
            'fees_24h': protocol.get('total24h', 0),     
            'fees_7d': protocol.get('total7d', 0),       
            'fees_30d': protocol.get('total30d', 0),     
            'fees_all_time': protocol.get('totalAllTime', 0),
            'data_type': 'fees',  # Add data type identifier
            'chain': 'solana',
            'timestamp': datetime.now()
        })
    
    # Convert to DataFrame
    df2 = pd.DataFrame(fees_list)
    
    # Display summary
    protocols_with_data = (df2['fees_24h'] > 0).sum()
    print(f"💰 Found fee data for {protocols_with_data} active protocols:")
    
    # Show top 10
    top_protocols = df2.sort_values(by="fees_24h", ascending=False).head(10)
    print(f"\n{'Protocol':<25} {'24h Fees':<15} {'7d Fees':<15} {'30d Fees':<15}")
    print("=" * 70)
    for _, row in top_protocols.iterrows():
        if row['fees_24h'] > 0:
            print(f"{row['protocol'][:24]:<25} {format_currency(row['fees_24h']):<15} "
                  f"{format_currency(row['fees_7d']):<15} {format_currency(row['fees_30d']):<15}")
    
    # Save DataFrame to joblib
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'solana_fees_{timestamp}.joblib'
    save_raw_data(df2, filename, 'Solana fees data')
    print(f"\n💾 DataFrame saved to {filename}")
    
    return df2


print("📊 Solana Protocol Fees Tracker")
print("=" * 50)
    
fees_df = get_solana_fees_data()
    
if fees_df is not None:
    print(f"\n✅ Successfully collected fees data for {len(fees_df)} protocols")
    print("Data saved to joblib file for further analysis")
else:
    print("\n❌ Failed to collect fees data")
    
print("\n" + "=" * 50)


🔍 Collecting Solana Protocol Fees Data...
📊 Solana Protocol Fees Tracker
✅ Collected Solana fees data
Total protocols found: 117
💰 Found fee data for 104 active protocols:

Protocol                  24h Fees        7d Fees         30d Fees       
Jupiter Perpetual Exchan  $3.14M          $22.95M         $94.84M        
PumpSwap                  $2.99M          $12.54M         $37.80M        
pump.fun                  $1.62M          $10.26M         $45.90M        
Axiom                     $1.56M          $10.65M         $53.68M        
Solana                    $1.24M          $9.06M          $42.67M        
Jito Liquid Staking       $1.06M          $4.13M          $12.50M        
Meteora DLMM              $988.57K        $7.56M          $53.84M        
Sanctum Validator LSTs    $905.43K        $3.64M          $12.86M        
Binance Staked SOL        $793.21K        $3.26M          $10.59M        
Raydium AMM               $645.47K        $5.37M          $24.34M        
💾 Saved raw:

#### Get all Solana tokens list via Jupiter API

In [7]:
# Fetch Jupiter token list
url = "https://token.jup.ag/all"
resp = requests.get(url)
tokens = resp.json()

# Convert to DataFrame
df = pd.DataFrame(tokens)

# Select useful columns
jupiter_df = df[["address", "symbol", "name", "decimals", "logoURI"]]

#### Step 4: Collect CoinGecko Price and Supply Data

In [ ]:
print("\n🔍 Collecting Enhanced CoinGecko Data for Protocols...")
print("=" * 50)

def get_coingecko_protocol_data(tvl_df):
    if tvl_df.empty:
        print("❌ No TVL data available for CoinGecko mapping.")
        return {}
    # Get all CoinGecko coins list
    coins_list_url = f"{API_ENDPOINTS['coingecko']['base_url']}/coins/list"
    coins_list = make_request(coins_list_url, headers={"x-cg-pro-api-key": API_KEYS['coingecko']})
    if not coins_list:
        print("❌ Failed to fetch CoinGecko coins list.")
        return {}
    coins_df = pd.DataFrame(coins_list)
    coins_df['name_lower'] = coins_df['name'].str.lower().str.strip()
    coins_df['symbol_lower'] = coins_df['symbol'].str.lower().str.strip()
    # Prepare TVL data for matching
    tvl_df['name_lower'] = tvl_df['name'].str.lower().str.strip()
    tvl_df['symbol_lower'] = tvl_df['symbol'].str.lower().str.strip()
    matched, unmatched = {}, []
    for _, row in tqdm(tvl_df.iterrows(), total=len(tvl_df), desc="Matching protocols to CoinGecko"):
        name_lower, symbol_lower = row['name_lower'], row['symbol_lower']
        # Match by name or symbol
        match = coins_df[
            (coins_df['name_lower'] == name_lower) |
            (coins_df['symbol_lower'] == symbol_lower)
        ]
        if not match.empty:
            coin_id = match.iloc[0]['id']
            # Fetch detailed coin data using /coins/{id} endpoint
            coin_url = f"{API_ENDPOINTS['coingecko']['base_url']}/coins/{coin_id}"
            headers = {"x-cg-pro-api-key": API_KEYS['coingecko']}
            params = {'localization': 'false', 'market_data': 'true'}  # Ensure market data is included
            coin_data = make_request(coin_url, headers=headers, params=params)
            if coin_data and 'market_data' in coin_data:
                matched[name_lower] = {
                    'protocol_name': row['name'],
                    'symbol': coin_data.get('symbol', '').upper(),
                    'current_price_usd': coin_data.get('market_data', {}).get('current_price', {}).get('usd', 0),
                    'market_cap_usd': coin_data.get('market_data', {}).get('market_cap', {}).get('usd', 0),
                    'price_change_24h_percent': coin_data.get('market_data', {}).get('price_change_percentage_24h', 0),
                    'tvl': row.get('tvl', 0),
                    'category': row.get('category', 'Unknown'),
                    'coingecko_id': coin_id,
                    'circulating_supply': coin_data.get('market_data', {}).get('circulating_supply', 0),  
                    'total_supply': coin_data.get('market_data', {}).get('total_supply', 0) 
                }
            time.sleep(1.0)  # Respect rate limits (Pro tier allows 100 calls/min, but cautious delay)
        else:
            unmatched.append(name_lower)
    if unmatched:
        print(f"⚠️ {len(unmatched)} protocols unmatched: {', '.join(unmatched[:5])}...")
    print(f"✅ Matched {len(matched)} protocols with CoinGecko data")
    save_raw_data(matched, f'solana_coingecko_enhanced_{datetime.now().strftime("%Y%m%d_%H%M%S")}.joblib', 'Enhanced CoinGecko protocol data')
    return matched

# Run collection
coingecko_data = get_coingecko_protocol_data(tvl_df)
print("Data saved to joblib file for further analysis" if coingecko_data else "\n❌ Failed to collect CoinGecko data")
print("\n" + "=" * 50)


🔍 Collecting Enhanced CoinGecko Data for Protocols...


Matching protocols to CoinGecko: 100%|██████████| 252/252 [06:14<00:00,  1.49s/it]

⚠️ 120 protocols unmatched: gauntlet, meteora dlmm, bouncebit cedefi yield, the vault liquid staking, superstate ustb...
✅ Matched 132 protocols with CoinGecko data
💾 Saved raw: Enhanced CoinGecko protocol data → ..\data\api_responses\solana_coingecko_enhanced_20250905_140727.joblib
Data saved to joblib file for further analysis



#### Step 5: Collect Helius Token Holder Data

In [9]:
print("\n🔍 Collecting Helius Token Holder Data for Solana DeFi Protocols...")
print("=" * 50)

helius_url = f"https://mainnet.helius-rpc.com/?api-key={API_KEYS['helius']}"
headers = {"Content-Type": "application/json"}

def get_solana_token_holders(jupiter_df, coingecko_data, tvl_df):
    # Basic input validation
    if not (coingecko_data and isinstance(coingecko_data, dict)):
        print("❌ No CoinGecko data available.")
        return pd.DataFrame()
    if jupiter_df.empty or 'address' not in jupiter_df.columns:
        print("❌ Jupiter token data missing or incomplete.")
        return pd.DataFrame()
    if tvl_df.empty or 'symbol' not in tvl_df.columns:
        print("❌ TVL data missing or incomplete.")
        return pd.DataFrame()

    # Map protocol names to token addresses
    token_map, unmatched = {}, []
    for key, data in coingecko_data.items():
        symbol = data.get('symbol', '').lower().strip()
        protocol = data.get('protocol_name', key)
        match = jupiter_df[jupiter_df['symbol'].str.lower() == symbol]
        if not match.empty:
            token_map[protocol] = match.iloc[0]['address']
        else:
            unmatched.append((protocol, symbol, data.get('coingecko_id', '')))
    if unmatched:
        print(f"⚠️ {len(unmatched)} tokens not found in Jupiter list: {', '.join([f'{n} ({s})' for n, s, _ in unmatched[:5]])}")
        save_cache(unmatched, f'unmatched_tokens_{datetime.now():%Y%m%d_%H%M%S}.joblib', "Unmatched tokens log")

    # Collect holders
    holders, logs = [], []
    for name, address in tqdm(token_map.items(), desc="Fetching token holders"):
        try:
            payload = {
                "jsonrpc": "2.0", 
                "id": "1", 
                "method": "getTokenLargestAccounts", 
                "params": [address]
                }
            resp = requests.post(helius_url, json=payload, headers=headers).json()
            accounts = resp.get('result', {}).get('value', [])[:10]
            if not accounts:
                logs.append(f"⚠️ No accounts found for {name}")
                continue
            # Symbol fallback
            symbol = coingecko_data.get(name.lower().replace(' ', '_'), {}).get('symbol', '')
            if not symbol:
                match = tvl_df[tvl_df['name'].str.lower() == name.lower()]
                symbol = match.iloc[0]['symbol'] if not match.empty else name
            # Add holder info
            for rank, acc in enumerate(accounts, 1):
                holders.append({
                    'token_name': name, 'token_symbol': symbol, 'token_address': address,
                    'rank': rank, 'account_address': acc['address'],
                    'ui_amount': acc.get('uiAmount', 0), 'raw_amount': acc.get('amount', '0'),
                    'decimals': acc.get('decimals', 0), 'timestamp': datetime.now()
                })
            logs.append(f"✅ {name}: {len(accounts)} accounts, top holder: {accounts[0].get('uiAmount', 0):,.0f} tokens")
            time.sleep(1.0)
        except Exception as e:
            logs.append(f"❌ Error processing {name}: {e}")

    save_cache(logs, f'token_holder_logs_{datetime.now():%Y%m%d_%H%M%S}.joblib', "Token holder collection logs")
    df = pd.DataFrame(holders)
    # Add percentage column
    if not df.empty:
        df['percentage_of_top10'] = df.groupby('token_name')['ui_amount'].transform(lambda x: x / x.sum() * 100 if x.sum() > 0 else 0)
    print(f"\n✅ Fetched data for {len(token_map)} tokens. Records: {len(df)}")
    save_raw_data(df, f'solana_token_holders_{datetime.now():%Y%m%d_%H%M%S}.joblib', 'Solana token holders DataFrame')
    return df

# Run collection

holders_data = get_solana_token_holders(jupiter_df, coingecko_data, tvl_df)
print("Data saved to joblib file for further analysis" if not holders_data.empty else "\n❌ Failed to collect token holders data")
print("\n" + "=" * 50)


🔍 Collecting Helius Token Holder Data for Solana DeFi Protocols...
⚠️ 14 tokens not found in Jupiter list: Maple (mpl), Unit ($unit), OpenEden TBILL (tbill), Apollo Diversified Credit Securitize Fund (acred), VanEck Treasury Fund (vbill)
💾 Saved cache: Unmatched tokens log → ..\data\temp\unmatched_tokens_20250905_140750.joblib


Fetching token holders: 100%|██████████| 118/118 [06:05<00:00,  3.10s/it]

💾 Saved cache: Token holder collection logs → ..\data\temp\token_holder_logs_20250905_141356.joblib

✅ Fetched data for 118 tokens. Records: 1038
💾 Saved raw: Solana token holders DataFrame → ..\data\api_responses\solana_token_holders_20250905_141356.joblib
Data saved to joblib file for further analysis

